In [ ]:
from model import DQN
import os
import minerl
import gym
import torch
import torch.nn.functional as F
import torch.optim as optim
import ray
import pandas as pd
import gc
import asyncio
from _collections import deque
from utils import *
import random
import copy


# learner에 샘플을 추가한다. 
# 
def learner_append_sample(memory, model, target_model, state, action, reward, next_state, done):
    # Caluclating Priority (TD Error)
    target = model(state.float()).data.cpu()
    old_val = target[0][action].cpu()
    target_val = target_model(next_state.float()).data.cpu()
    if done:
        target[0][action] = reward
    else:
        target[0][action] = reward + 0.99 * torch.max(target_val)

    error = abs(old_val - target[0][action])
    error = error.cpu()
    memory.add.remote(error, [state, action, reward, next_state, done])



In [3]:
import numpy as np
a = np.array(([1, 1, 1]))
c = np.power(a, 2)

In [5]:
b = a * 1/3
b

array([0.33333333, 0.33333333, 0.33333333])

In [6]:
c = np.power(b, 2)
print(c)

[0.11111111 0.11111111 0.11111111]


In [ ]:
@ray.remote
class Actor:
    def __init__(self, learner, actor_idx, startEpsilon, endEpsilon, paramServer):
        # environment initialization
        self.actor_idx = actor_idx
        self.env = gym.make("MineRLTreechop-v0")
        self.port_number = int("12340") + actor_idx
        print("actor environment %d initialize successfully" % self.actor_idx)
        self.env.make_interactive(port=self.port_number, realtime=False)
        self.shared_network_cpu = ray.get(learner.get_network.remote())
        # self.shared_memory = ray.get(shared_memory_id)
        # print("shared memory assign successfully")
        
        # network initalization
        self.actor_network = DQN(19).cpu()
        self.actor_target_network = DQN(19).cpu()
        self.actor_network.load_state_dict(self.shared_network_cpu.state_dict())
        self.actor_target_network.load_state_dict(self.actor_network.state_dict())
        print("actor network %d initialize successfully" % self.actor_idx)

        self.initialized = False
        self.epi_counter = 0
        # exploring info
        self.startEpsilon = startEpsilon
        self.endEpsilon = endEpsilon
        self.max_episodes = 1000

        self.paramServer = paramServer
        
    
    # 1. 네트워크 파라미터 복사
    # 2. 환경 탐험 (초기화, 행동)
    # 3. 로컬버퍼에 저장
    # 4. priority 계산
    # 5. 글로벌 버퍼에 저장
    # 6. 주기적으로 네트워크 업데이트

    def get_initialized(self):
        return self.initialized

    def get_counter(self):
        return self.epi_counter

    # 각 환경 인스턴스에서 각 엡실론에 따라 탐험을 진행한다.
    # 탐험 과정에서 local buffer에 transition들을 저장한다.
    # local buffer의 개수가 특정 개수 이상이면 global buffer에 추가해준다.

    def explore(self, learner, shared_memory):
        
        self.initialized = True
        stepDrop = (self.startEpsilon - self.endEpsilon) / self.max_episodes
        epsilon = self.startEpsilon
        total_steps = 0
        
        episodes = [x for x in range(self.max_episodes)]
        train_stats = pd.DataFrame(index=episodes, columns=['rewards'])
        
        for num_epi in range(self.max_episodes):
            obs = self.env.reset()
            state = converter(obs).cpu().float()
            done = False
            total_reward = 0
            steps = 0
            if (epsilon > self.endEpsilon):
                epsilon -= stepDrop
                
            while not done:
                steps += 1
                total_steps += 1
                a_out = self.actor_network.sample_action(state, epsilon)
                action_index = a_out
                action = make_action(self.env, action_index)
                obs_prime, reward, done, info = self.env.step(action)
                total_reward += reward
                state_prime = converter(obs_prime)

                self.actor_append_sample(shared_memory, self.actor_network, self.actor_target_network, \
                                       state, action_index, reward, state_prime, done)

                state = state_prime.float().cpu()
                if done:
                    break
                    
            # pandas로 리워드 기록하기
            print("%d episode is done" % num_epi)
            print("total rewards : %d " % total_reward)
            train_stats.loc[num_epi]['rewards'] = total_reward
            train_stats.to_csv('train_stat_minerl_agent {}.csv'.format(str(self.actor_idx)))
            
  
            self.pull_parameters(learner) 
            print("actor network is updated ")
            print("actor target_network is updated")
    
    def pull_parameters(self, learner):
        ray.get(self.paramServer.pull_parameters.remote(learner)) 
        policy_params, target_params = ray.get(self.paramServer.return_parameters.remote())
        self.actor_network.load_state_dict(policy_params)
        self.actor_target_network.load_state_dict(target_params)
        
    def env_close(self):
        self.env.close()        

    def actor_append_sample(self, memory, model, target_model, state, action, reward, next_state, done):
        # Caluclating Priority (TD Error)
        target = model(state.float()).data.cpu()
        old_val = target[0][action].cpu()
        target_val = target_model(next_state.float()).data.cpu()
        if done:
            target[0][action] = reward
        else:
            target[0][action] = reward + 0.99 * torch.max(target_val)

        error = abs(old_val - target[0][action])
        error = error.cpu()
        memory.add.remote(error, [state, action, reward, next_state, done])






In [ ]:
@ray.remote
class ParameterServer:
    def __init__(self):
        self.policy_params = DQN(19).state_dict()
        self.target_params = DQN(19).state_dict()
    
    def pull_parameters(self, learner):
        learner.push_parameters.remote(self.policy_params, self.target_params)
        return 1
    
    def return_parameters(self):
        return self.policy_params, self.target_params
    

In [ ]:
@ray.remote(num_gpus=1)
class Learner:
    def __init__(self, network, batch_size, paramServer):
        self.learner_network = DQN(19).cuda().float()
        self.learner_target_network = DQN(19).cuda().float()
        self.learner_network.load_state_dict(network.state_dict())
        self.learner_target_network.load_state_dict(network.state_dict())
        self.shared_network = DQN(19).cpu()
        self.shared_target_network = DQN(19).cpu()
        
        self.paramServer = paramServer
        
        self.count = 0
        self.batch_size = batch_size
        self.max_counts= 1000000

    # 1. sampling
    # 2. calculate gradient
    # 3. weight update
    # 4. compute priorities
    # 5. priorities of buffer update
    # 6. remove old memory
    
    def push_parameters(self, server_policy_params, server_target_params):
        self.shared_network.load_state_dict(self.learner_network.state_dict())
        self.shared_target_network.load_state_dict(self.learner_target_network.state_dict())
        policy_net_params = self.shared_network.state_dict()
        target_net_params = self.shared_target_network.state_dict()
        server_policy_params = policy_net_params
        server_target_params = target_net_params
        
    def count(self):
        return self.count
    
    def get_network(self):
        self.shared_network.load_state_dict(self.learner_network.state_dict())
        print("return learner network")
        return self.shared_network
    
    def get_target_network(self):
        self.shared_target_network.load_state_dict(self.learner_target_network.state_dict())
        return self.shared_target_network

    def update_network(self, memory, demos, batch_size, optimizer):
        print("started")

        
        counts = [x for x in range(self.max_counts)]
        train_stats = pd.DataFrame(index=counts, columns=['loss'])
        while(self.count < 1000000):
           

            agent_batch, agent_idxs, agent_weights = ray.get(memory.sample.remote(batch_size))
            demo_batch, demo_idxs, demo_weights = ray.get(demos.sample.remote(batch_size))

            # demo_batch = (batch_size, state, action, reward, next_state, done, n_rewards)
            # print(len(demo_batch[0])) # 0번째 배치이므로 0이 나옴
            state_list = []
            action_list = []
            reward_list = []
            next_state_list = []
            done_mask_list = []

            #print("agent batch len : {} ".format(str(len(agent_batch))))
            for agent_transition in agent_batch:
                s, a, r, s_prime, done_mask = agent_transition
                state_list.append(s)
                action_list.append([a])
                reward_list.append([r])
                next_state_list.append(s_prime)
                done_mask_list.append([done_mask])

            #print("demo batch len : {} ".format(str(len(demo_batch))))
            
            for expert_transition in demo_batch:
                s, a, r, s_prime, done_mask = expert_transition
                state_list.append(s)
                action_list.append([a])
                reward_list.append([r])
                next_state_list.append(s_prime)
                done_mask_list.append([done_mask])

            s = torch.stack(state_list).float().cuda()
            a = torch.tensor(action_list, dtype=torch.int64).cuda()
            r = torch.tensor(reward_list).cuda()
            s_prime = torch.stack(next_state_list).float().cuda()
            done_mask = torch.tensor(done_mask_list).float().cuda()

            q_vals = self.learner_network(s)
            state_action_values = q_vals.gather(1, a)

            # comparing the q values to the values expected using the next states and reward
            next_state_values = self.learner_target_network(s_prime).max(1)[0].unsqueeze(1)
            target = r + (next_state_values * gamma * done_mask)

            # calculating the q loss, n-step return lossm supervised_loss
            is_weights = torch.FloatTensor(agent_weights).to(device)
            q_loss = (is_weights * F.mse_loss(state_action_values, target)).mean()
            #supervised_loss = margin_loss(q_vals, a, 1, 1)

            loss = q_loss #+ supervised_loss
            errors = torch.abs(state_action_values - target).data.cpu().detach()
            errors = errors.numpy()
            # update priority
            for i in range(batch_size):
                idx = agent_idxs[i]
                memory.update.remote(idx, errors[i])

            train_stats.loc[self.count ]['loss'] = float(loss.item())
            train_stats.to_csv('train_stat_minerl_learner.csv')

            # optimization step and logging
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            self.count +=1
            if(self.count % 50 == 0 and self.count != 0):
                self.learner_target_network.load_state_dict(self.learner_network.state_dict())
                print("Count : {} leaner_target_network updated".format(self.count))
                
            if(self.count % 10 == 0 and self.count!= 0):
                print("Count : {} leaner_network updated".format(self.count))
                torch.save(self.learner_network.state_dict(), model_path + "apex_dqfd_learner.pth")
                print("learner model saved")
                




In [ ]:
@ray.remote
def parse_demo(env_name, rep_buffer, policy_net, target_net, threshold=10, num_epochs=1, batch_size=16,
      seq_len=10, gamma=0.99):
    data = minerl.data.make(env_name)
    print("data loading sucess")
    demo_num = 0
    parse_ts = 0

    for s_batch, a_batch, r_batch, ns_batch, d_batch in data.batch_iter(num_epochs=num_epochs, batch_size=batch_size,seq_len=seq_len):
        print(ray.get(rep_buffer.size.remote()))
        if(ray.get(rep_buffer.size.remote()) > 9999):
            del data
            return

        demo_num += 1
        print(demo_num)
        print(r_batch.sum())
        if r_batch.sum() < threshold:
            del s_batch, a_batch, r_batch, d_batch, ns_batch
            continue


        batch_length = (s_batch['pov'].shape)[0]  # (batch, seq, 64, 64, 3)[0]
        for i in range(0, batch_length):
            episode_start_ts = 0

            for j in range(0, seq_len):
                av = a_batch['attack'][i][j]  # attack value
                aj = a_batch['jump'][i][j]  # jump value
                af = a_batch['forward'][i][j]  # forward value
                ab = a_batch['back'][i][j]  # back value
                al = a_batch['left'][i][j]  # left value
                ar = a_batch['right'][i][j]  # right value
                va = a_batch['camera'][i][j][0]  # vertical angle and
                ha = a_batch['camera'][i][j][1]  # horizontal angle

                camera_thresholds = (abs(va) + abs(ha)) / 2.0
                # 카메라를 움직이는 경우
                if (camera_thresholds > 2.5):
                    # camera = [0, -5]
                    if abs(va) < abs(ha) and ha < 0:
                        if av == 0:
                            action_index = 0
                        else:
                            action_index = 1
                    # camera = [0, 5]
                    elif abs(va) < abs(ha) and ha > 0:
                        if av == 0:
                            action_index = 2
                        else:
                            action_index = 3
                    # camera = [-5, 0]
                    elif abs(va) > abs(ha) and ha < 0:
                        if av == 0:
                            action_index = 4
                        else:
                            action_index = 5
                    # camera = [5, 0]
                    elif abs(va) > abs(ha) and ha > 0:
                        if av == 0:
                            action_index = 6
                        else:
                            action_index = 7

                            # 카메라를 안움직이는 경우
                # 점프하는 경우
                elif (aj == 1):
                    if (af == 0):
                        action_index = 8
                    else:
                        action_index = 9

                # 앞으로 가는 경우
                elif (af == 1):
                    if (av == 0):
                        action_index = 10
                    else:
                        action_index = 11

                # 뒤로 가는 경우
                elif (ab == 1):
                    if (av == 0):
                        action_index = 12
                    else:
                        action_index = 13

                # 왼쪽으로 가는 경우
                elif (al == 1):
                    if (av == 0):
                        action_index = 14
                    else:
                        action_index = 15

                # 오른쪽으로 가는 경우
                elif (ar == 1):
                    if (av == 0):
                        action_index = 16
                    else:
                        action_index = 17

                # 카메라, 움직임이 다 0이고 공격만 하는 것
                else:
                    if (av == 0):
                        continue
                    else:
                        action_index = 18

                a_index = torch.LongTensor([action_index]).cpu()
                state = converter2(s_batch['pov'][i][j]).float().cpu()
                next_state = converter2(ns_batch['pov'][i][j]).float().cpu()
                reward = torch.FloatTensor([r_batch[i][j]]).cpu()
                done = d_batch[i][j]  # .astype(int)



                learner_append_sample(rep_buffer, policy_net, target_net, state, a_index, reward, next_state, done)
                episode_start_ts += 1
                parse_ts += 1

                # if episode done we reset
                if done:
                    break
            print('{} expert samples added.'.format(episode_start_ts))

        gc.collect()
        print('Batch Parsed finished. {} expert samples added.'.format(parse_ts))


In [ ]:
ray.init()

#하이퍼 파라미터
learning_rate = 0.0003
gamma = 0.99
buffer_limit = 50000
L1 = 0.9
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

root_path = os.curdir
model_path = root_path + '/dqn_model/'

In [ ]:
policy_net = DQN(19).cuda()
target_net = DQN(19).cuda()
target_net.load_state_dict(policy_net.state_dict())
memory = Memory.remote(30000)
demos = Memory.remote(10000)
optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate, weight_decay=1e-5)

# Copy network params from pretrained Agent
model_path = './dqn_model/per_dqn.pth'
policy_net.load_state_dict(torch.load(model_path, map_location='cuda:0'))
target_net.load_state_dict(policy_net.state_dict())

#parse_demo2.remote("MineRLTreechop-v0", demos, policy_net.cpu(), target_net.cpu(), optimizer, threshold=60, num_epochs=1, batch_size=4, seq_len=60, gamma=0.99, model_name='pre_trained4.pth')


In [ ]:
params_server = ParameterServer.remote()

In [ ]:
# learner network initialzation
batch_size = 256
learner = Learner.remote(policy_net, batch_size, params_server)

In [ ]:
# actor network, environments initialization
# Generating each own instances

actor1 = Actor.remote(learner, 0, 0.95, 0.05, params_server)
actor2 = Actor.remote(learner, 1, 0.5, 0.025, params_server)
actor3 = Actor.remote(learner, 2, 0.05, 0.01, params_server)
#actor_list = [actor1, actor2]
actor_list = [actor1, actor2, actor3]

In [ ]:
#parse = parse_demo.remote("MineRLTreechop-v0", demos, ray.get(learner.get_network.remote()), ray.get(learner.get_target_network.remote()), threshold=40, num_epochs=1, batch_size=4,
#              seq_len=400, gamma=0.99)


In [ ]:
explore = [actor.explore.remote(learner, memory) for actor in actor_list]


In [ ]:
update = learner.update_network.remote(memory, demos, batch_size, optimizer)
